In [ ]:

import torch
from transformers import GPT2Tokenizer, GPT2Model, GPT2PreTrainedModel, AdamW
from torch import nn
from sklearn.metrics import accuracy_score
from datasets import load_dataset
from torch.utils.data import DataLoader , TensorDataset, Dataset
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define MLP to setup on top of GPT2

class TextMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim,  linear = False):
        super(TextMLP, self).__init__()

        if linear != True: 
            self.seq = nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, output_dim)
            )

        else:
            self.seq = nn.Sequential(
                nn.Linear(input_dim, output_dim),
            )

    def forward(self, x):
        return self.seq(x)



def get_imdb_dataset():

    dataset = load_dataset('imdb', split=['train', 'test'])
    return dataset[0], dataset[1] # train_dataset, test_dataset

def createDataloader(encoder_dataset):
    labels = torch.tensor(encoder_dataset['label'])

    tensor_ids = torch.tensor(encoder_dataset['input_ids'])
    tensor_att = torch.tensor(encoder_dataset['attention_mask'])
    data = torch.cat([tensor_ids, tensor_att], dim = 1).to(torch.long)

    dataset_ = TensorDataset(data , labels)
    
    return DataLoader(dataset_, batch_size = 32)

train_data, test_data = get_imdb_dataset()

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token



# Encode the dataset
#encoded_dataset_train = train_data.map(lambda data: tokenizer(data['text'], truncation=True, padding='max_length', return_tensors="pt"))
#encoded_dataset_test = test_data.map(lambda data: tokenizer(data['text'], truncation=True, padding='max_length'), batched=True)

   


In [ ]:
print(train_data[0])

9


In [ ]:
encoded_dataset_train[0]

In [ ]:

# Load the GPT-2 model and Freeze GPT-2 parameters
gpt2_model = GPT2Model.from_pretrained("gpt2")

for param in gpt2_model.parameters():
    param.requires_grad = False

advtestloader = createDataloader(encoded_dataset_train)

a, l = next(iter(advtestloader))
a = a.to(device)
gpt2_model = gpt2_model.to(device)

with torch.no_grad():
    o = gpt2_model(input_ids = a[:,0,:],attention_mask = a[:,1,:])